数据加载代码块


In [1]:
# ============== 数据加载代码块 ==============
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息

# 尝试读取数据文件
try:
    df = pd.read_csv('data/messy_data.csv')
    print("成功加载数据文件")
except FileNotFoundError:
    # 如果文件不存在，则从create_messy_data导入创建函数
    from create_messy_data import create_messy_data
    df = create_messy_data()
    print("创建了新的示例数据")

# 显示数据基本信息
print("\n=== 数据基本信息 ===")
print(f"数据维度: {df.shape}")              # 显示行数和列数
print(f"内存占用: {df.memory_usage().sum() / 1024**2:.2f} MB")  # 显示内存占用
print("\n列名列表:")
print(df.columns.tolist())                  # 显示所有列名

# 显示数据预览
print("\n=== 数据预览（前5行）===")
print(df.head())

# 显示数据类型信息
print("\n=== 数据类型信息 ===")
print(df.dtypes)

成功加载数据文件

=== 数据基本信息 ===
数据维度: (100020, 13)
内存占用: 9.92 MB

列名列表:
['日期', '姓名', '年龄', '工资', '邮箱', '手机号', '地址', '部门', '产品编码', '订单金额', '客户评分', '物流时效', '评价数量']

=== 数据预览（前5行）===
           日期  姓名  年龄     工资                     邮箱          手机号  \
0  2250-04-28  周九  34  14524  user83027@example.com  18959860363   
1  2213-04-08  孙八  49  17158  user69493@example.com  17622100137   
2  2127-01-03  赵六  37  11144  user37987@example.com  13739734261   
3  2108-12-05  钱七  27  14011  user31384@example.com  17442871760   
4  2080-02-19  吴十  20  19377  user20868@example.com  15924341524   

             地址   部门     产品编码     订单金额  客户评分  物流时效 评价数量  
0   北京市朝阳区建国路8号  市场部  PRD1355  1177.46  85.1  32.8  105  
1  北京市朝阳区建国路57号  销售部  PRD9110  1699.05  79.5   6.8  123  
2  北京市朝阳区建国路41号  技术部  PRD3624    765.2  77.1  30.4  111  
3  北京市朝阳区建国路17号  技术部  PRD6859  6267.07  81.2  59.3  118  
4  北京市朝阳区建国路28号  销售部  PRD2479  4716.91  84.8  36.2  104  

=== 数据类型信息 ===
日期      object
姓名      object
年龄      object
工资      o

缺失值审核

In [11]:


# 改进的缺失值检测函数
def is_special_null(value):
    """识别特殊的空值标记"""
    special_nulls = [
        'N/A', 'NA', 'null', 'NULL', 'none', 'NONE', '', 'invalid_date',
        'invalid_age', 'invalid_salary', 'pending', '待审核', '未评分',
        '待确认', 'error', 'hidden', '未知'
    ]
    # 检测是否为 NaN 或是否在特殊标记列表中
    return pd.isnull(value) or str(value).strip() in special_nulls

# 1. 在转换之前，检测每列中有多少特殊空值标记
print("=== 特殊空值标记检测（转换前） ===")
for column in df.columns:
    special_null_count = df[column].apply(lambda x: is_special_null(x)).sum()
    print(f"{column} 列: 特殊空值标记数量 = {special_null_count}")

# 2. 将自定义空值标记转换为 NaN
df = df.applymap(lambda x: np.nan if is_special_null(x) else x)

# 3. 重新计算缺失值统计
missing_stats = pd.DataFrame({
    '缺失值数量': df.isnull().sum(),
    '缺失值比例': (df.isnull().sum() / len(df) * 100).round(2),
    '数据类型': df.dtypes
})

print("\n=== 缺失值统计（转换后） ===")
print(missing_stats)


=== 特殊空值标记检测（转换前） ===
日期 列: 特殊空值标记数量 = 1
姓名 列: 特殊空值标记数量 = 0
年龄 列: 特殊空值标记数量 = 3
工资 列: 特殊空值标记数量 = 4
邮箱 列: 特殊空值标记数量 = 0
手机号 列: 特殊空值标记数量 = 2
地址 列: 特殊空值标记数量 = 3
部门 列: 特殊空值标记数量 = 0
产品编码 列: 特殊空值标记数量 = 3
订单金额 列: 特殊空值标记数量 = 4
客户评分 列: 特殊空值标记数量 = 3
物流时效 列: 特殊空值标记数量 = 2
评价数量 列: 特殊空值标记数量 = 4

=== 缺失值统计（转换后） ===
      缺失值数量  缺失值比例    数据类型
日期        1   0.10  object
姓名        0   0.00  object
年龄        3   0.29  object
工资        4   0.39  object
邮箱        0   0.00  object
手机号       2   0.20  object
地址        3   0.29  object
部门        0   0.00  object
产品编码      3   0.29  object
订单金额      4   0.39  object
客户评分      3   0.29  object
物流时效      2   0.20  object
评价数量      4   0.39  object


缺失值处理

In [12]:
# ============== 缺失值处理代码块 ==============
# 创建数据副本进行处理
df_cleaned = df.copy()

# 1. 处理数值列中的异常数据和缺失值
numeric_columns = ['年龄', '工资', '订单金额', '客户评分', '物流时效', '评价数量']  # 明确指定数值列
for col in numeric_columns:
    print(f"\n处理 {col} 列:")
    # 1.1 将非数值数据转换为NaN
    # 先将列转换为字符串
    df_cleaned[col] = df_cleaned[col].astype(str)
    # 使用数值模式匹配，保留合法的数值（包括负数和小数）
    mask = df_cleaned[col].str.match(r'^-?\d*\.?\d+$')
    df_cleaned.loc[~mask, col] = np.nan
    # 将列转换为float类型
    df_cleaned[col] = df_cleaned[col].astype(float)
    
    # 1.2 使用中位数填充缺失值
    median_value = df_cleaned[col].median()
    df_cleaned[col] = df_cleaned[col].fillna(median_value)
    print(f"  - 已将非数值数据转换为NaN并用中位数 {median_value:.2f} 填充")
    
    # 1.3 显示处理结果
    print(f"  - 列的唯一值: {df_cleaned[col].unique()[:5]}...")  # 只显示前5个唯一值
    print(f"  - 剩余缺失值数量: {df_cleaned[col].isnull().sum()}")

# 2. 处理分类列的缺失值
categorical_columns = ['部门', '产品编码']
for col in categorical_columns:
    # 使用众数填充分类列的缺失值
    mode_value = df_cleaned[col].mode()[0]
    df_cleaned[col] = df_cleaned[col].fillna(mode_value)
    print(f"\n列 '{col}' 的缺失值已用众数 '{mode_value}' 填充")

# 3. 处理文本列的特殊空值
text_columns = ['姓名', '地址', '邮箱', '手机号']
for col in text_columns:
    # 将特殊空值标记转换为NaN
    df_cleaned[col] = df_cleaned[col].replace(['N/A', 'NA', 'null', 'NULL', 'none', 'NONE', ''], np.nan)
    # 对于关键信息，可以选择删除包含缺失值的行
    if col in ['姓名', '手机号']:  # 假设这些是必需的字段
        df_cleaned = df_cleaned.dropna(subset=[col])
        print(f"已删除 '{col}' 列中包含缺失值的行")

# 4. 验证处理结果
print("\n=== 缺失值处理后的统计 ===")
missing_after = pd.DataFrame({
    '处理后缺失值数量': df_cleaned.isnull().sum(),
    '处理后缺失值比例': (df_cleaned.isnull().sum() / len(df_cleaned) * 100).round(2)
})
print(missing_after)

# 5. 数据类型转换
# 将数值列转换为适当的类型
for col in numeric_columns:
    if col in ['年龄', '评价数量']:  # 应该是整数的列
        df_cleaned[col] = df_cleaned[col].round().astype('Int64')
    else:  # 可以是小数的列
        df_cleaned[col] = df_cleaned[col].astype('float64')

# 显示处理前后的数据量对比
print(f"\n处理前数据行数: {len(df)}")
print(f"处理后数据行数: {len(df_cleaned)}")

# 6. 保存处理后的数据
# 确保输出目录存在
import os
if not os.path.exists('data/processed'):
    os.makedirs('data/processed')

# 保存处理后的数据到新文件
output_file = 'data/processed/data_after_missing_handled.csv'
df_cleaned.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n处理后的数据已保存至: {output_file}")

# 7. 生成处理报告
report = f"""
========== 缺失值处理报告 ==========
处理时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

1. 数据规模变化:
   - 原始数据行数: {len(df)}
   - 处理后数据行数: {len(df_cleaned)}
   - 减少行数: {len(df) - len(df_cleaned)}

2. 处理方法说明:
   - 数值列: 先转换非数值为NaN，然后用中位数填充
   - 分类列: 使用众数填充
   - 文本列: 特殊空值转换为NaN，关键字段行删除

3. 数据类型转换:
   - 年龄和评价数量: 转换为整数
   - 其他数值列: 保留为浮点数

4. 处理结果文件位置:
   {output_file}

5. 各列缺失值处理结果:
{missing_after.to_string()}
"""

# 保存处理报告
report_file = 'data/processed/missing_values_handling_report.txt'
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(report)
print(f"\n处理报告已保存至: {report_file}")

# 8. 显示处理后的数据样本
print("\n=== 处理后的数据样本（前5行）===")
print(df_cleaned.head())


处理 年龄 列:
  - 已将非数值数据转换为NaN并用中位数 41.00 填充
  - 列的唯一值: [20. 51. 29. 41. 31.]...
  - 剩余缺失值数量: 0

处理 工资 列:
  - 已将非数值数据转换为NaN并用中位数 12564.00 填充
  - 列的唯一值: [9422. 5130. 7708. 9569. 9473.]...
  - 剩余缺失值数量: 0

处理 订单金额 列:
  - 已将非数值数据转换为NaN并用中位数 3011.11 填充
  - 列的唯一值: [6201.96 2565.66 5685.1  2607.32 3173.57]...
  - 剩余缺失值数量: 0

处理 客户评分 列:
  - 已将非数值数据转换为NaN并用中位数 85.40 填充
  - 列的唯一值: [86.8 82.7 81.4 89.3 92.7]...
  - 剩余缺失值数量: 0

处理 物流时效 列:
  - 已将非数值数据转换为NaN并用中位数 16.80 填充
  - 列的唯一值: [ 4.9 18.1 38.2 18.2 25.7]...
  - 剩余缺失值数量: 0

处理 评价数量 列:
  - 已将非数值数据转换为NaN并用中位数 99.00 填充
  - 列的唯一值: [ 93. 106. 111. 114.  91.]...
  - 剩余缺失值数量: 0

列 '部门' 的缺失值已用众数 '技术部' 填充

列 '产品编码' 的缺失值已用众数 'PRD6136' 填充
已删除 '姓名' 列中包含缺失值的行
已删除 '手机号' 列中包含缺失值的行

=== 缺失值处理后的统计 ===
      处理后缺失值数量  处理后缺失值比例
日期           1      0.10
姓名           0      0.00
年龄           0      0.00
工资           0      0.00
邮箱           0      0.00
手机号          0      0.00
地址           3      0.29
部门           0      0.00
产品编码         0      0.00
订单金额         0    

一致性审核

In [13]:
# ============== 一致性审核代码块 ==============
import pandas as pd
import numpy as np
import re
from datetime import datetime

print("=== 开始数据一致性审核 ===")

# 1. 日期格式一致性审核
print("\n1. 日期格式审核:")
date_patterns = df_cleaned['日期'].str.extract(r'(\d{4})[-/.](\d{1,2})[-/.](\d{1,2})')
invalid_dates = df_cleaned[date_patterns.isnull().any(axis=1)]['日期']
print(f"发现 {len(invalid_dates)} 个不规范的日期格式:")
print(invalid_dates.head())

# 2. 名称格式一致性审核
print("\n2. 姓名格式审核:")
# 检查中文姓名格式
chinese_name_pattern = re.compile(r'^[\u4e00-\u9fa5]{2,4}$')
invalid_names = df_cleaned[~df_cleaned['姓名'].str.match(chinese_name_pattern, na=False)]
print(f"发现 {len(invalid_names)} 个不规范的姓名格式:")
print(invalid_names['姓名'].head())

# 3. 手机号格式一致性审核
print("\n3. 手机号格式审核:")
phone_pattern = re.compile(r'^1[3-9]\d{9}$')
invalid_phones = df_cleaned[~df_cleaned['手机号'].str.match(phone_pattern, na=False)]
print(f"发现 {len(invalid_phones)} 个不规范的手机号格式:")
print(invalid_phones['手机号'].head())

# 4. 邮箱格式一致性审核
print("\n4. 邮箱格式审核:")
email_pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
invalid_emails = df_cleaned[~df_cleaned['邮箱'].str.match(email_pattern, na=False)]
print(f"发现 {len(invalid_emails)} 个不规范的邮箱格式:")
print(invalid_emails['邮箱'].head())

# 5. 部门名称一致性审核
print("\n5. 部门名称审核:")
dept_variants = df_cleaned['部门'].value_counts()
print("当前部门名称变体:")
print(dept_variants)

# 6. 产品编码一致性审核
print("\n6. 产品编码审核:")
code_pattern = re.compile(r'^PRD\d{4}$')
invalid_codes = df_cleaned[~df_cleaned['产品编码'].str.match(code_pattern, na=False)]
print(f"发现 {len(invalid_codes)} 个不规范的产品编码:")
print(invalid_codes['产品编码'].head())

# 7. 生成一致性审核报告
consistency_report = {
    '日期格式问题': len(invalid_dates),
    '姓名格式问题': len(invalid_names),
    '手机号格式问题': len(invalid_phones),
    '邮箱格式问题': len(invalid_emails),
    '产品编码问题': len(invalid_codes),
    '部门名称变体数': len(dept_variants)
}

print("\n=== 一致性审核总结 ===")
for item, count in consistency_report.items():
    print(f"{item}: {count}")

=== 开始数据一致性审核 ===

1. 日期格式审核:
发现 6 个不规范的日期格式:
137    01/19/2023
181       2023-01
254    01-15-2023
314      20230117
587           NaN
Name: 日期, dtype: object

2. 姓名格式审核:
发现 10 个不规范的姓名格式:
41     ZHANG SAN
199          王五 
250        李   四
301           张三
345          李四 
Name: 姓名, dtype: object

3. 手机号格式审核:
发现 8 个不规范的手机号格式:
72     086-13888888888
128        abc12345678
164    +86-13888888888
183      138 8888 8888
196         2345678901
Name: 手机号, dtype: object

4. 邮箱格式审核:
发现 8 个不规范的邮箱格式:
18           @example.com
68                  user@
430              user@com
539     user@example.com 
938             user@.com
Name: 邮箱, dtype: object

5. 部门名称审核:
当前部门名称变体:
技术部          266
销售部          261
人事部          251
市场部          230
人事部门           1
销售部门           1
Tech           1
HR             1
SALES          1
Sales          1
技术部门           1
市场部门           1
Marketing      1
sales          1
Name: 部门, dtype: int64

6. 产品编码审核:
发现 7 个不规范的产品编码:
219    PRD_1234
319     Prd1234
458    

一致性处理

In [14]:
# ============== 一致性处理代码块 ==============
# 创建数据副本
df_standardized = df_cleaned.copy()

print("=== 开始数据一致性处理 ===")

# 1. 标准化日期格式
def standardize_date(date_str):
    if pd.isna(date_str):
        return None
    # 尝试各种日期格式
    try:
        # 先尝试直接解析
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except:
        # 如果解析失败，返回None
        return None

print("\n1. 处理日期格式:")
df_standardized['日期'] = df_standardized['日期'].apply(standardize_date)
print("日期格式已标准化为 YYYY-MM-DD")

# 2. 标准化姓名格式
def standardize_name(name):
    if pd.isna(name):
        return None
    # 去除空白字符
    name = re.sub(r'\s+', '', str(name))
    # 转换为中文名称格式（如果是拼音）
    name_mapping = {
        'zhang san': '张三', 'li si': '李四', 'wang wu': '王五',
        'zhao liu': '赵六', 'qian qi': '钱七'
    }
    return name_mapping.get(name.lower(), name)

print("\n2. 处理姓名格式:")
df_standardized['姓名'] = df_standardized['姓名'].apply(standardize_name)
print("姓名格式已标准化")

# 3. 标准化手机号格式
def standardize_phone(phone):
    if pd.isna(phone):
        return None
    # 提取数字
    digits = re.sub(r'\D', '', str(phone))
    # 如果是11位手机号，且以1开头
    if len(digits) == 11 and digits.startswith('1'):
        return digits
    return None

print("\n3. 处理手机号格式:")
df_standardized['手机号'] = df_standardized['手机号'].apply(standardize_phone)
print("手机号格式已标准化")

# 4. 标准化邮箱格式
def standardize_email(email):
    if pd.isna(email):
        return None
    # 去除首尾空白
    email = str(email).strip().lower()
    # 检查基本邮箱格式
    if re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email):
        return email
    return None

print("\n4. 处理邮箱格式:")
df_standardized['邮箱'] = df_standardized['邮箱'].apply(standardize_email)
print("邮箱格式已标准化")

# 5. 标准化部门名称
department_mapping = {
    'Sales': '销售部', 'SALES': '销售部', 'sales': '销售部', '销售部门': '销售部',
    'Tech': '技术部', 'TECH': '技术部', '技术部门': '技术部',
    'Marketing': '市场部', 'MARKETING': '市场部', '市场部门': '市场部',
    'HR': '人事部', 'Human Resources': '人事部', '人事部门': '人事部'
}

print("\n5. 处理部门名称:")
df_standardized['部门'] = df_standardized['部门'].replace(department_mapping)
print("部门名称已标准化")

# 6. 标准化产品编码
def standardize_product_code(code):
    if pd.isna(code):
        return None
    # 提取数字部分
    digits = re.search(r'\d{4}', str(code))
    if digits:
        return f"PRD{digits.group()}"
    return None

print("\n6. 处理产品编码:")
df_standardized['产品编码'] = df_standardized['产品编码'].apply(standardize_product_code)
print("产品编码已标准化")

# 7. 验证处理结果
print("\n=== 验证处理结果 ===")
for column in ['日期', '姓名', '手机号', '邮箱', '部门', '产品编码']:
    print(f"\n{column}的唯一值:")
    print(df_standardized[column].value_counts().head())

# 8. 保存标准化后的数据
output_file = 'data/processed/data_after_standardization.csv'
df_standardized.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n标准化后的数据已保存至: {output_file}")

# 9. 生成处理报告
standardization_report = f"""
========== 数据标准化处理报告 ==========
处理时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

1. 数据标准化范围:
   - 日期格式: 统一为 YYYY-MM-DD
   - 姓名格式: 统一为中文姓名
   - 手机号格式: 统一为11位数字
   - 邮箱格式: 统一为小写，去除空格
   - 部门名称: 统一为中文部门名
   - 产品编码: 统一为PRD+4位数字

2. 处理结果统计:
   日期格式标准化: {df_standardized['日期'].notna().sum()} 条有效记录
   姓名格式标准化: {df_standardized['姓名'].notna().sum()} 条有效记录
   手机号格式标准化: {df_standardized['手机号'].notna().sum()} 条有效记录
   邮箱格式标准化: {df_standardized['邮箱'].notna().sum()} 条有效记录
   部门名称标准化: {df_standardized['部门'].notna().sum()} 条有效记录
   产品编码标准化: {df_standardized['产品编码'].notna().sum()} 条有效记录

3. 数据规模:
   处理前记录数: {len(df_cleaned)}
   处理后记录数: {len(df_standardized)}

4. 处理结果文件位置:
   {output_file}
"""

# 保存处理报告
report_file = 'data/processed/standardization_report.txt'
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(standardization_report)
print(f"\n标准化处理报告已保存至: {report_file}")

=== 开始数据一致性处理 ===

1. 处理日期格式:
日期格式已标准化为 YYYY-MM-DD

2. 处理姓名格式:
姓名格式已标准化

3. 处理手机号格式:
手机号格式已标准化

4. 处理邮箱格式:
邮箱格式已标准化

5. 处理部门名称:
部门名称已标准化

6. 处理产品编码:
产品编码已标准化

=== 验证处理结果 ===

日期的唯一值:
2023-01-01    4
2023-03-24    2
2023-09-19    2
2023-01-15    2
2024-02-24    2
Name: 日期, dtype: int64

姓名的唯一值:
周九    147
钱七    138
李四    132
吴十    125
赵六    124
Name: 姓名, dtype: int64

手机号的唯一值:
18989136502    2
15122955324    2
18291673280    2
15689220202    2
15887822880    2
Name: 手机号, dtype: int64

邮箱的唯一值:
user82@example.com     2
user294@example.com    2
user419@example.com    2
user944@example.com    2
user772@example.com    2
Name: 邮箱, dtype: int64

部门的唯一值:
技术部    268
销售部    265
人事部    253
市场部    232
Name: 部门, dtype: int64

产品编码的唯一值:
PRD1234    7
PRD6136    6
PRD8380    3
PRD4085    2
PRD2077    2
Name: 产品编码, dtype: int64

标准化后的数据已保存至: data/processed/data_after_standardization.csv

标准化处理报告已保存至: data/processed/standardization_report.txt


异常值审核

In [15]:
# ============== 异常值审核代码块 ==============
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

print("=== 开始异常值审核 ===")

# 1. 数值列异常值检测
# 需要检查异常值的数值列
numeric_columns = ['年龄', '工资', '订单金额', '客户评分', '物流时效', '评价数量']

# 创建异常值报告字典
outlier_report = {}

for column in numeric_columns:
    print(f"\n检查 {column} 的异常值:")
    
    # 计算基本统计量
    stats_data = {
        '平均值': df_standardized[column].mean(),
        '中位数': df_standardized[column].median(),
        '标准差': df_standardized[column].std(),
        '最小值': df_standardized[column].min(),
        '最大值': df_standardized[column].max()
    }
    
    # 计算四分位数
    Q1 = df_standardized[column].quantile(0.25)
    Q3 = df_standardized[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # 定义异常值边界
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 找出异常值
    outliers = df_standardized[
        (df_standardized[column] < lower_bound) | 
        (df_standardized[column] > upper_bound)
    ]
    
    # 收集异常值信息
    outlier_report[column] = {
        '统计量': stats_data,
        '四分位数': {'Q1': Q1, 'Q3': Q3, 'IQR': IQR},
        '异常值边界': {'下界': lower_bound, '上界': upper_bound},
        '异常值数量': len(outliers),
        '异常值比例': (len(outliers) / len(df_standardized) * 100),
        '异常值示例': outliers[column].head().tolist()
    }
    
    # 打印异常值检测结果
    print(f"基本统计量:")
    for k, v in stats_data.items():
        print(f"  {k}: {v:.2f}")
    print(f"\n异常值边界:")
    print(f"  下界: {lower_bound:.2f}")
    print(f"  上界: {upper_bound:.2f}")
    print(f"\n发现 {len(outliers)} 条异常记录 ({(len(outliers)/len(df_standardized)*100):.2f}%)")
    
    # 绘制箱线图
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=df_standardized[column])
    plt.title(f"{column} 的箱线图")
    plt.savefig(f'data/processed/boxplot_{column}.png')
    plt.close()

# 2. 业务规则异常检测
print("\n=== 业务规则异常检测 ===")

# 2.1 检查年龄范围
age_violations = df_standardized[
    (df_standardized['年龄'] < 18) | 
    (df_standardized['年龄'] > 70)
]
print(f"\n年龄异常（<18 或 >70）: {len(age_violations)} 条")

# 2.2 检查工资范围
salary_violations = df_standardized[
    (df_standardized['工资'] < 3000) | 
    (df_standardized['工资'] > 50000)
]
print(f"工资异常（<3000 或 >50000）: {len(salary_violations)} 条")

# 2.3 检查评分范围
score_violations = df_standardized[
    (df_standardized['客户评分'] < 0) | 
    (df_standardized['客户评分'] > 100)
]
print(f"评分异常（<0 或 >100）: {len(score_violations)} 条")

# 2.4 检查物流时效异常
delivery_violations = df_standardized[
    (df_standardized['物流时效'] < 0) | 
    (df_standardized['物流时效'] > 72)  # 假设正常配送不超过72小时
]
print(f"物流时效异常（<0 或 >72小时）: {len(delivery_violations)} 条")

# 3. 生成异常值审核报告
print("\n=== 生成异常值审核报告 ===")

audit_report = f"""
========== 异常值审核报告 ==========
生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

1. 统计异常检测结果：
"""

for column, report in outlier_report.items():
    audit_report += f"""
{column}:
    - 异常值数量: {report['异常值数量']}
    - 异常值比例: {report['异常值比例']:.2f}%
    - 值域范围: [{report['统计量']['最小值']:.2f}, {report['统计量']['最大值']:.2f}]
    - 异常值边界: [{report['异常值边界']['下界']:.2f}, {report['异常值边界']['上界']:.2f}]
    - 异常值示例: {report['异常值示例']}
"""

audit_report += """
2. 业务规则违规检测结果：
"""
audit_report += f"""
    - 年龄异常: {len(age_violations)} 条
    - 工资异常: {len(salary_violations)} 条
    - 评分异常: {len(score_violations)} 条
    - 物流时效异常: {len(delivery_violations)} 条
"""

# 保存审核报告
report_file = 'data/processed/outlier_audit_report.txt'
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(audit_report)
print(f"\n异常值审核报告已保存至: {report_file}")

=== 开始异常值审核 ===

检查 年龄 的异常值:
基本统计量:
  平均值: 41.14
  中位数: 41.00
  标准差: 32.88
  最小值: -1.00
  最大值: 999.00

异常值边界:
  下界: -1.50
  上界: 82.50

发现 3 条异常记录 (0.29%)

检查 工资 的异常值:
基本统计量:
  平均值: 12424.79
  中位数: 12564.00
  标准差: 4305.54
  最小值: -5000.00
  最大值: 19925.00

异常值边界:
  下界: -2870.62
  上界: 27820.38

发现 1 条异常记录 (0.10%)

检查 订单金额 的异常值:
基本统计量:
  平均值: 4557.58
  中位数: 3011.11
  标准差: 31313.63
  最小值: -50.00
  最大值: 1000000.00

异常值边界:
  下界: -1575.25
  上界: 8113.30

发现 45 条异常记录 (4.42%)

检查 客户评分 的异常值:
基本统计量:
  平均值: 84.83
  中位数: 85.40
  标准差: 10.13
  最小值: -10.00
  最大值: 150.00

异常值边界:
  下界: 58.05
  上界: 112.05

发现 6 条异常记录 (0.59%)

检查 物流时效 的异常值:
基本统计量:
  平均值: 20.07
  中位数: 16.80
  标准差: 15.55
  最小值: -2.00
  最大值: 240.00

异常值边界:
  下界: -14.90
  上界: 51.50

发现 33 条异常记录 (3.24%)

检查 评价数量 的异常值:
基本统计量:
  平均值: 1081.80
  中位数: 99.00
  标准差: 31338.81
  最小值: -1.00
  最大值: 999999.00

异常值边界:
  下界: 73.50
  上界: 125.50

发现 7 条异常记录 (0.69%)

=== 业务规则异常检测 ===

年龄异常（<18 或 >70）: 5 条
工资异常（<3000 或 >50000）: 1 条
评分异常（<0 或 >100）: 2 条
物流时效异常（<0 或

In [16]:
# 创建数据副本
df_outliers_handled = df_standardized.copy()

print("=== 开始异常值处理 ===")

# 1. 基于业务规则的异常值处理
print("\n1. 处理业务规则异常")

# 1.1 处理年龄异常
print("\n处理年龄异常:")
age_mask = (df_outliers_handled['年龄'] >= 18) & (df_outliers_handled['年龄'] <= 70)
invalid_age = ~age_mask
if invalid_age.any():
    median_age = int(df_outliers_handled.loc[age_mask, '年龄'].median())  # Convert to int
    df_outliers_handled.loc[invalid_age, '年龄'] = median_age
    print(f"已将 {invalid_age.sum()} 个异常年龄值替换为中位数: {median_age}")

# 1.2 处理工资异常
print("\n处理工资异常:")
salary_mask = (df_outliers_handled['工资'] >= 3000) & (df_outliers_handled['工资'] <= 50000)
invalid_salary = ~salary_mask
if invalid_salary.any():
    median_salary = int(df_outliers_handled.loc[salary_mask, '工资'].median())  # Convert to int
    df_outliers_handled.loc[invalid_salary, '工资'] = median_salary
    print(f"已将 {invalid_salary.sum()} 个异常工资值替换为中位数: {median_salary}")

# 1.3 处理评分异常
print("\n处理评分异常:")
score_mask = (df_outliers_handled['客户评分'] >= 0) & (df_outliers_handled['客户评分'] <= 100)
invalid_score = ~score_mask
if invalid_score.any():
    median_score = int(df_outliers_handled.loc[score_mask, '客户评分'].median())  # Convert to int
    df_outliers_handled.loc[invalid_score, '客户评分'] = median_score
    print(f"已将 {invalid_score.sum()} 个异常评分值替换为中位数: {median_score}")

# 1.4 处理物流时效异常
print("\n处理物流时效异常:")
delivery_mask = (df_outliers_handled['物流时效'] >= 0) & (df_outliers_handled['物流时效'] <= 72)
invalid_delivery = ~delivery_mask
if invalid_delivery.any():
    median_delivery = int(df_outliers_handled.loc[delivery_mask, '物流时效'].median())  # Convert to int
    df_outliers_handled.loc[invalid_delivery, '物流时效'] = median_delivery
    print(f"已将 {invalid_delivery.sum()} 个异常物流时效值替换为中位数: {median_delivery}")

# 2. 基于统计的异常值处理
print("\n2. 处理统计异常值")

for column in numeric_columns:
    # 获取列的数据类型
    is_integer = pd.api.types.is_integer_dtype(df_outliers_handled[column].dtype)
    
    # 计算四分位数
    Q1 = df_outliers_handled[column].quantile(0.25)
    Q3 = df_outliers_handled[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # 定义异常值边界
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 如果是整数类型，将边界值转换为整数
    if is_integer:
        lower_bound = int(np.ceil(lower_bound))  # 向上取整
        upper_bound = int(np.floor(upper_bound))  # 向下取整
    
    # 找出异常值
    outliers_mask = (df_outliers_handled[column] < lower_bound) | (df_outliers_handled[column] > upper_bound)
    outliers_count = outliers_mask.sum()
    
    if outliers_count > 0:
        # 使用缩尾处理：将异常值设置为边界值
        df_outliers_handled.loc[df_outliers_handled[column] < lower_bound, column] = lower_bound
        df_outliers_handled.loc[df_outliers_handled[column] > upper_bound, column] = upper_bound
        print(f"\n{column}: 已处理 {outliers_count} 个异常值（缩尾处理）")

=== 开始异常值处理 ===

1. 处理业务规则异常

处理年龄异常:
已将 5 个异常年龄值替换为中位数: 41

处理工资异常:
已将 1 个异常工资值替换为中位数: 12564

处理评分异常:
已将 2 个异常评分值替换为中位数: 85

处理物流时效异常:
已将 8 个异常物流时效值替换为中位数: 16

2. 处理统计异常值

订单金额: 已处理 45 个异常值（缩尾处理）

客户评分: 已处理 4 个异常值（缩尾处理）

物流时效: 已处理 27 个异常值（缩尾处理）

评价数量: 已处理 7 个异常值（缩尾处理）
